## SAM segmentation, meristem delineation, size and curvature estimation & signal quantification

### Necessary imports

In [ ]:
import os
from time import time as current_time
from copy import deepcopy

import numpy as np
import scipy.ndimage as nd
import pandas as pd
import matplotlib.pyplot as plt

from scipy.optimize import linear_sum_assignment

import panel
import vtk

from timagetk.components import SpatialImage, LabelledImage
from timagetk.io import imread, imsave
from timagetk.plugins.resampling import isometric_resampling
from timagetk.plugins import labels_post_processing

from cellcomplex.utils.array_dict import array_dict

from cellcomplex.property_topomesh.creation import tetrahedra_topomesh, triangle_topomesh, edge_topomesh, vertex_topomesh
from cellcomplex.property_topomesh.analysis import compute_topomesh_property, compute_topomesh_vertex_property_from_faces, topomesh_property_gaussian_filtering
from cellcomplex.property_topomesh.morphology import topomesh_binary_property_morphological_operation
from cellcomplex.property_topomesh.optimization import property_topomesh_vertices_deformation
from cellcomplex.property_topomesh.extraction import property_filtering_sub_topomesh, topomesh_connected_components
from cellcomplex.property_topomesh.io import read_ply_property_topomesh, save_ply_property_topomesh
   
from cellcomplex.property_topomesh.visualization.vtk_actor_topomesh import VtkActorTopomesh

from cellcomplex.property_topomesh.utils.matching_tools import kd_tree_match
from cellcomplex.property_topomesh.utils.geometry_tools import triangle_geometric_features, tetra_geometric_features
 
from tissue_nukem_3d.microscopy_images.read_microscopy_image import read_czi_image, read_tiff_image
from tissue_nukem_3d.nuclei_mesh_tools import nuclei_image_surface_topomesh, up_facing_surface_topomesh
from tissue_nukem_3d.utils.matplotlib_tools import draw_box

from tissue_analysis.tissue_analysis import TissueImage
from tissue_analysis.array_tools import find_geometric_median
from tissue_analysis.property_spatial_image import PropertySpatialImage

from sam_atlas.meristem_surface import meristem_surface_topomesh, curvature_meristem_extraction

from visu_core.matplotlib import glasbey
from visu_core.matplotlib.colormap import plain_colormap

from visu_core.vtk.display import vtk_display_notebook, vtk_display_image_notebook, vtk_save_screenshot_actors
from visu_core.vtk.volume import vtk_image_volume
from visu_core.vtk.actor import vtk_actor

from visu_core.vtk.utils.image_tools import vtk_image_data_from_image, image_to_vtk_cell_polydatas, image_to_surface_intensity_polydata, image_to_surface_cell_scalar_property_polydata
from visu_core.vtk.utils.polydata_tools import vtk_combine_polydatas, vtk_tube_polydata, vtk_slice_polydata

### Import locally defined tools

In [ ]:
from utils.image_tools import membrane_image_segmentation, labelled_image_projection
from utils.property_image_tools import image_property_morphological_operation, property_image_to_dataframe, image_property_median_filtering
from utils.adjacency_tools import triangles_from_adjacency_edges

### File path information

In [ ]:
dirname = '/Users/gcerutti/Projects/RDP/LayerHeight_Marketa/test/'
#dirname = '/home/carlos/Documents/WORK/Image analysis/20160625 MS-E3 CLV-DR5/LD/RAW'

#filename = 'E15_SAM02'
filename = 'E_set_blue_SAM02'
#filename = 'Col-0-PI-SAM3'
#filename = 'CLV3-CH-DR5-3VE-MS-E3-LD-SAM3'
#filename = 'CLV3-CH-DR5-3VE-MS-E3-LD-SAM5'

### Algorithm parameters

In [ ]:
# image parameters
channel_names = ['PI']
#channel_names = ['Signal','PI']
#channel_names = ['CLV3','DR5','PI']
membrane_channel = 'PI'
#signal_channels = ['CLV3','DR5']
#signal_channels = ['Signal']
signal_channels = []
microscope_orientation = 1

# segmentation parameters
recompute_segmentation = False
gaussian_sigma = 0.75
h_min = 200
segmentation_gaussian_sigma = 0.5
volume_threshold = 100000

# SAM surface extraction parameters
recompute_surface = False
resampling_voxelsize = 1.
gaussian_curvature_threshold = -2.5e-4
min_curvature_threshold = -1e-3

# Layer height estimation parameters
surface_matching_distance_threshold = 20.

# Signal quantification parameters
quantification_erosion_radius = 1.

# Visualization parameters
subsampling = 2
figure_size = 8

### Preparing folder architecture

In [ ]:
if not os.path.exists(dirname + "/" + filename + "/"):
    os.makedirs(dirname + "/" + filename + "/")

### Reading the microscopy image

In [ ]:
microscopy_filename = dirname + '/' + filename + '.czi'
img_dict = read_czi_image(microscopy_filename, channel_names)
if len(channel_names) == 1:
    img_dict = {channel_names[0]:img_dict}

In [ ]:
actors = []
image_data = vtk_image_data_from_image(img_dict[membrane_channel].get_array(), 
                                       voxelsize=img_dict[membrane_channel].voxelsize)
volume = vtk_image_volume(image_data,colormap='gray')
actors += [volume]

figure_filename = dirname + '/' + filename + '/' + filename + '_membrane_image.png'
vtk_save_screenshot_actors(actors, figure_filename, focal_point=(0,0,-microscope_orientation), view_up=(-microscope_orientation,0,0))

#actors = []
#surface_polydata = image_to_surface_intensity_polydata(img_dict[membrane_channel],subsampling=3,threshold=2000)
#surface_actor = vtk_actor(surface_polydata,colormap='gray')
#actors += [surface_actor]

vtk_display_notebook(actors)
#vtk_display_image_notebook(image_data,colormap='Greys')


### Auto-seeded watershed segmentation

In [ ]:
segmentation_filename = dirname + '/' + filename + '/' + filename + '_' + membrane_channel + '_seg.tif'

if recompute_segmentation or not os.path.exists(segmentation_filename):
    seg_img = membrane_image_segmentation(img_dict[membrane_channel],
                                          gaussian_sigma=gaussian_sigma,
                                          h_min=h_min,
                                          segmentation_gaussian_sigma=segmentation_gaussian_sigma,
                                          volume_threshold=volume_threshold)
    imsave(segmentation_filename, seg_img)
else:
    seg_img = read_tiff_image(segmentation_filename)
    seg_img = LabelledImage(seg_img, no_label_id=0)


In [ ]:
actors = []

labels = [l for l in seg_img.labels() if l!=1]

cell_polydatas = image_to_vtk_cell_polydatas(seg_img,
                                             labels=labels, 
                                             subsampling=[int(np.round(0.8/v)) for v in seg_img.voxelsize], 
                                             smoothing=10)
cell_polydata = vtk_combine_polydatas([cell_polydatas[l] for l in labels])
cell_actor = vtk_actor(cell_polydata,colormap='glasbey',value_range=(0,255))
actors += [cell_actor]

figure_filename = dirname + '/' + filename + '/' + filename + '_segmented_cells.png'
vtk_save_screenshot_actors(actors, figure_filename, focal_point=(0,0,-microscope_orientation), view_up=(-microscope_orientation,0,0))
vtk_display_notebook(actors)

### Surface extraction

In [ ]:
binary_img = SpatialImage(nd.gaussian_filter(255*(seg_img>1),sigma=1./np.array(seg_img.voxelsize)).astype(np.uint8),voxelsize=seg_img.voxelsize)

if resampling_voxelsize is not None:
    binary_img = isometric_resampling(binary_img, method=resampling_voxelsize, option='linear')
else:
    resampling_voxelsize = np.array(binary_img.voxelsize)

In [ ]:
surface_filename = dirname + '/' + filename + '/' + filename + '_' + membrane_channel + '_surface.ply'

curvature_properties = ['mean_curvature', 'gaussian_curvature', 'principal_curvature_min', 'principal_curvature_max']

if recompute_surface or not os.path.exists(surface_filename):
    topomesh = meristem_surface_topomesh(signal_img=None,
                                         seg_img=seg_img,
                                         resampling_voxelsize=resampling_voxelsize,
                                         microscope_orientation=microscope_orientation)
    properties_to_save = {}
    properties_to_save[0] = curvature_properties + ['normal','barycenter_z']
    properties_to_save[1] = []
    properties_to_save[2] = curvature_properties + ['normal','area']
    properties_to_save[3] = []

    save_ply_property_topomesh(topomesh,surface_filename,properties_to_save)
else:
    topomesh = read_ply_property_topomesh(surface_filename)

In [ ]:
actors = []

surface_actor = VtkActorTopomesh(topomesh,2,property_name='principal_curvature_min',property_degree=0)
surface_actor.update(colormap='RdBu_r',value_range=(-0.1,0.1),opacity=1)
actors += [surface_actor]

figure_filename = dirname + '/' + filename + '/' + filename + '_surface_min_curvature.png'
vtk_save_screenshot_actors(actors, figure_filename, focal_point=(0,0,-microscope_orientation), view_up=(-microscope_orientation,0,0))
vtk_display_notebook(actors)

### Curvature-based SAM delineation

In [ ]:
meristem_topomesh = curvature_meristem_extraction(topomesh,
                                                  curvature_property='principal_curvature_min',
                                                  curvature_threshold=min_curvature_threshold,
                                                  image_center=np.array(seg_img.extent)/2.)

meristem_area = meristem_topomesh.wisp_property('area',2).values(list(meristem_topomesh.wisps(2))).sum()
meristem_radius = np.sqrt(meristem_area/np.pi)

meristem_curvature = np.nanmedian(meristem_topomesh.wisp_property('mean_curvature',2).values(list(meristem_topomesh.wisps(2))))

In [ ]:
actors = []

# actor = VtkActorTopomesh(topomesh,2,property_name='gaussian_curvature',property_degree=0)
surface_actor = VtkActorTopomesh(topomesh,2,property_name='principal_curvature_min',property_degree=0)
# actor.update(colormap='RdBu_r',value_range=(-0.002,0.002))
surface_actor.update(colormap='RdBu_r',value_range=(-0.1,0.1),opacity=0.1)
# surface_actor.update(colormap='RdBu_r',value_range=(-0.1,0.1),opacity=1)
actors += [surface_actor]

# actor = VtkActorTopomesh(topomesh,2,property_name='gaussian_curvature',property_degree=0)
actor = VtkActorTopomesh(meristem_topomesh,2,property_name='principal_curvature_min',property_degree=0)
# actor.update(colormap='RdBu_r',value_range=(-0.002,0.002))
actor.update(colormap='RdBu_r',value_range=(-0.1,0.1))
actors += [actor]


figure_filename = dirname + '/' + filename + '/' + filename + '_meristem_surface.png'
vtk_save_screenshot_actors(actors, figure_filename, focal_point=(0,0,-microscope_orientation), view_up=(-microscope_orientation,0,0))
vtk_display_notebook(actors)

### Meristem projection on the segmented image

In [ ]:
p_img = PropertySpatialImage(seg_img)
p_img.compute_image_property('layer')

In [ ]:
cell_layer = p_img.image_property('layer')
all_layer_img = p_img.create_property_image('layer', background_value=0)

surface_voxelsize = 0.66

layer_img = {}
cell_surface_center = {}
for layer in [1,2,3,4]:
    layer_img[layer] = LabelledImage(SpatialImage((all_layer_img.get_array()>layer-1).astype(np.uint8),voxelsize=seg_img.voxelsize),no_label_id=0)
    
    layer_seg_img = deepcopy(seg_img)
    layer_seg_img[layer_img[layer]==0] = 1
    
    if surface_voxelsize is not None:
        layer_seg_img = isometric_resampling(layer_seg_img, method=surface_voxelsize, option='label')
    else:
        surface_voxelsize = np.array(seg_img.voxelsize)
    layer_seg_img = LabelledImage(SpatialImage(layer_seg_img,voxelsize=surface_voxelsize),no_label_id=0)
    
    layer_tissue_img = TissueImage(layer_seg_img, no_label_id=0, background=1)
    elements_coord = layer_tissue_img.topological_elements(element_order=[2])
    for d in [2]:
        for k in elements_coord[d].keys():
            elements_coord[d][k] *= surface_voxelsize
    
    layer_cells = [c for c in p_img.labels if cell_layer[c]==layer]
    for c in layer_cells:
        if (1,c) in elements_coord[2].keys():
            surface_center = elements_coord[2][(1,c)][find_geometric_median(elements_coord[2][(1,c)])]
            cell_surface_center[c] = surface_center
        else:
            print("Missed cell",c)

cell_surface_center = array_dict(cell_surface_center)

In [ ]:
cell_surface_topomesh = vertex_topomesh(cell_surface_center)
cell_surface_topomesh.update_wisp_property('layer',0,{c:cell_layer[c] for c in cell_surface_center.keys()})

actors = []

cell_actor = VtkActorTopomesh(cell_surface_topomesh, 0, property_name='layer', point_glyph='sphere')
cell_actor.update(colormap='jet',value_range=(0,4))
actors += [cell_actor]

vtk_display_notebook(actors)

In [ ]:
cell_center = p_img.image_property('barycenter')
cell_layer = p_img.image_property('layer')

cell_points = cell_center.values(p_img.labels)
meristem_points = meristem_topomesh.wisp_property('barycenter',0).values(list(meristem_topomesh.wisps(0)))
cell_meristem_distances = np.linalg.norm(cell_points[:,np.newaxis] - meristem_points[np.newaxis],axis=2).min(axis=1)
cell_meristem_distances = array_dict(cell_meristem_distances,keys=p_img.labels)

cell_meristem = {c:(cell_meristem_distances[c]<5.*resampling_voxelsize and cell_layer[c]==1) for c in p_img.labels}
p_img.update_image_property('meristem',cell_meristem)

image_property_morphological_operation(p_img,'meristem',method='closing',iterations=3)

cell_meristem = p_img.image_property('meristem')

layer_meristem_cell_tetra_heights = {}
layer_meristem_tetrahedra_meshes = {}

for layer in [2,3,4]:
    layer_cells = np.array([l for l in p_img.labels if cell_layer[l]==layer])
    prev_layer_cells = np.array([l for l in p_img.labels if cell_layer[l]==layer-1])
    
    prev_layer_cell_neighbors = np.array([n for l in prev_layer_cells 
                                            for n in p_img.image_graph.neighbors(l) 
                                            if n in prev_layer_cells])
    prev_layer_cell_neighbor_cells = np.array([l for l in prev_layer_cells 
                                                 for n in p_img.image_graph.neighbors(l) 
                                                 if n in prev_layer_cells])
    
    prev_layer_neighborhood_edges = np.transpose([prev_layer_cell_neighbor_cells,prev_layer_cell_neighbors])
    prev_layer_neighborhood_edges = np.unique(np.sort(prev_layer_neighborhood_edges),axis=0)
    
    prev_layer_neighborhood_triangles = triangles_from_adjacency_edges(prev_layer_neighborhood_edges)
    prev_layer_neighborhood_meristem_triangles = prev_layer_neighborhood_triangles[np.all(cell_meristem.values(prev_layer_neighborhood_triangles),axis=1)]
    
    prev_layer_meristem_tetrahedras = np.array([list(t)+[n] 
                                                for t in prev_layer_neighborhood_meristem_triangles
                                                for n in layer_cells 
                                                if np.all([n in p_img.image_graph.neighbors(l) for l in t])])
    
    valid_tetras = np.all(np.isin(prev_layer_meristem_tetrahedras,list(cell_surface_center.keys())),axis=1)
    prev_layer_meristem_tetrahedras = prev_layer_meristem_tetrahedras[valid_tetras]
    
    tetra_topomesh = tetrahedra_topomesh(prev_layer_meristem_tetrahedras,cell_surface_center)
    layer_meristem_tetrahedra_meshes[layer-1] = tetra_topomesh
    
    prev_layer_meristem_tetra_volumes = tetra_geometric_features(prev_layer_meristem_tetrahedras,cell_surface_center,['volume'])[:,0]
    prev_layer_meristem_tetra_base_areas = triangle_geometric_features(prev_layer_meristem_tetrahedras[:,:3],cell_surface_center,['area'])[:,0]
    prev_layer_meristem_tetra_heights = 3*prev_layer_meristem_tetra_volumes/prev_layer_meristem_tetra_base_areas
    layer_meristem_cell_tetra_heights[layer-1] = prev_layer_meristem_tetra_heights
    
    layer_meristem_cells = np.unique([t[-1] for t in prev_layer_meristem_tetrahedras])
                
    for c in layer_meristem_cells:
        cell_meristem[c] = True
    p_img.update_image_property('meristem',cell_meristem)
    image_property_morphological_operation(p_img,'meristem',method='closing',iterations=3,layer_restriction=None)
    cell_meristem = p_img.image_property('meristem')

In [ ]:
layer_colormaps = dict(zip([1,2,3,4],['Blues','Reds','Greens','Greys']))

actors = []
for layer in [1,2,3]:
    actor = VtkActorTopomesh(layer_meristem_tetrahedra_meshes[layer],3)
    actor.update_polydata()
    # actor.polydata = vtk_slice_polydata(vtk_slice_polydata(actor.polydata,axis='x',position=seg_img.extent[0]/2.,width=30.),axis='y',position=seg_img.extent[1]/2.,width=30.)
    # actor.display_polydata = vtk_slice_polydata(actor.polydata,axis='y',position=3*seg_img.extent[1]/5.,width=15.)
    actor.update_actor(colormap=layer_colormaps[layer],value_range=(0,1))
    actor.GetProperty().SetRepresentationToWireframe()
    actors += [actor]

figure_filename = dirname + '/' + filename + '/' + filename + '_inter_layer_tetrahedra.png'
#vtk_save_screenshot_actors([layer_actor], figure_filename, focal_point=(0,0,-microscope_orientation), view_up=(-microscope_orientation,0,0))
vtk_save_screenshot_actors(actors, figure_filename, focal_point=(0,1,0), view_up=(0,0,microscope_orientation))

vtk_display_notebook(actors)

In [ ]:
layer_colors = dict(zip([1,2,3,4],['b','r','g','k']))

figure = plt.figure(3)
figure.clf()
for layer in [1,2,3]:
    draw_box(figure,layer_meristem_cell_tetra_heights[layer],box_x=layer,box_width=0.33,color=layer_colors[layer],outlier_size=5)
figure.gca().set_xlim(0,4)
figure.gca().set_xticks([1,2,3])
figure.gca().set_xticklabels(['L'+str(l) for l in [1,2,3]],size=24)
figure.gca().set_ylim(0,surface_matching_distance_threshold)
figure.gca().set_ylabel("Layer height ($\mu$m)",size=24)
figure.set_size_inches(10,10)

boxplot_filename = dirname + '/' + filename + '/' + filename + '_layer_tetra_height_boxplot.png'
figure.savefig(boxplot_filename)

In [ ]:
cell_layer = p_img.image_property('layer')
cell_meristem = p_img.image_property('meristem')

#labels = [l for l in p_img.labels if p_img.image_property('barycenter')[l][1]>3*seg_img.extent[0]/5]
labels = [l for l in p_img.labels]

meristem_polydatas = image_to_vtk_cell_polydatas(seg_img,
                                                 labels=labels, 
                                                 cell_property=cell_meristem,
                                                 cell_polydatas=cell_polydatas)
meristem_polydata = vtk_combine_polydatas([meristem_polydatas[l] for l in labels])
meristem_actor = vtk_actor(meristem_polydata,colormap='glasbey',value_range=(-1,254))

figure_filename = dirname + '/' + filename + '/' + filename + '_meristem_cells.png'
vtk_save_screenshot_actors([meristem_actor], figure_filename, focal_point=(0,0,-microscope_orientation), view_up=(-microscope_orientation,0,0))
meristem_panel = vtk_display_notebook([meristem_actor])

labels = [l for l in p_img.labels if p_img.image_property('barycenter')[l][1]>3*seg_img.extent[0]/5
                                  and cell_meristem[l]]


layer_polydatas = image_to_vtk_cell_polydatas(seg_img,
                                              labels=labels, 
                                              cell_property=cell_layer,
                                              cell_polydatas=cell_polydatas)
layer_polydata = vtk_combine_polydatas([layer_polydatas[l] for l in labels])
layer_actor = vtk_actor(layer_polydata,colormap='jet',value_range=(0,4))

figure_filename = dirname + '/' + filename + '/' + filename + '_meristem_cell_layers.png'
vtk_save_screenshot_actors([layer_actor], figure_filename, focal_point=(0,1,0), view_up=(0,0,microscope_orientation))
layer_panel = vtk_display_notebook([layer_actor])

panel.Row(meristem_panel,layer_panel)

### Layer height estimation

In [ ]:
all_layer_img = p_img.create_property_image('layer', background_value=0)

surface_voxelsize = 2.

layer_img = {}
for layer in [1, 2, 3, 4]:
    layer_img[layer] = LabelledImage(SpatialImage((all_layer_img.get_array()>layer-1).astype(np.uint8),voxelsize=seg_img.voxelsize),no_label_id=0)

if surface_voxelsize is not None:
    for layer in [1, 2, 3, 4]:
        layer_img[layer] = isometric_resampling(layer_img[layer], method=surface_voxelsize, option='label')
else:
    surface_voxelsize = np.array(seg_img.voxelsize)

In [ ]:
surface_meshes = {}
for layer in [1, 2, 3, 4]:
    binary_img = nd.binary_erosion(layer_img[layer], border_value=1).astype(np.uint8)
    
    image_data = vtk_image_data_from_image(binary_img, voxelsize=layer_img[layer].voxelsize)

    contour = vtk.vtkImageMarchingCubes()
    contour.SetInputData(image_data)
    contour.ComputeNormalsOn()
    contour.SetValue(0, 0.5)
    contour.Update()

    print("--> Marching cubes :", contour.GetOutput().GetNumberOfPoints())

    normal_generator = vtk.vtkPolyDataNormals()
    normal_generator.SetInputData(contour.GetOutput())
    # normal_generator.SetInputData(smoother.GetOutput())
    normal_generator.ComputePointNormalsOn()
    normal_generator.ComputeCellNormalsOff()
    normal_generator.Update()

    polydata = normal_generator.GetOutput()
    surface_points = np.array([polydata.GetPoints().GetPoint(p) for p in range(polydata.GetPoints().GetNumberOfPoints())])
    surface_triangles = np.array([[polydata.GetCell(t).GetPointIds().GetId(i) for i in range(3)] for t in range(polydata.GetNumberOfCells())])
    surface_normals = np.array(polydata.GetPointData().GetArray('Normals'))

    unique_points = np.unique(surface_points,axis=0)
    print("--> Unique points :", len(unique_points))

    unique_point_matching = kd_tree_match(surface_points,unique_points,radius=1e-5)
    unique_triangles = unique_point_matching[surface_triangles]

    unique_normals = np.transpose([nd.sum(surface_normals[:,k],unique_point_matching,index=range(len(unique_points))) for k in range(3)])
    unique_normals = unique_normals/np.linalg.norm(unique_normals,axis=1)[:,np.newaxis]

    surface_topomesh = triangle_topomesh(unique_triangles,unique_points)
    surface_topomesh.update_wisp_property('normal', 0, dict(enumerate(unique_normals)))

    start_time = current_time()
    print("--> Smoothing surface mesh")
    property_topomesh_vertices_deformation(surface_topomesh, iterations=10, omega_forces={'laplacian_smoothing': 0.33})
    print("<-- Smoothing surface mesh [", current_time() - start_time, "s]")
    
    surface_meshes[layer] = surface_topomesh

In [ ]:
for layer in [1, 2, 3, 4]:
    surface_topomesh = surface_meshes[layer]
    
    surface_points = surface_topomesh.wisp_property('barycenter',0).values(list(surface_topomesh.wisps(0)))
    surface_normals = surface_topomesh.wisp_property('normal',0).values(list(surface_topomesh.wisps(0)))
    
    normal_points = surface_points
    
    normal_coords = np.round(normal_points/seg_img.voxelsize)
    normal_coords = np.maximum(np.minimum(normal_coords, np.array(seg_img.shape) - 1), 0)
    normal_coords[np.isnan(normal_coords)] = 0
    normal_coords = normal_coords.astype(int)
    normal_coords = tuple(np.transpose(normal_coords))
    
    surface_labels = seg_img.get_array()[normal_coords]
    surface_meristem = p_img.image_property('meristem').values(surface_labels)

    surface_topomesh.update_wisp_property('meristem',0,dict(zip(surface_topomesh.wisps(0),surface_meristem)))
    
    #topomesh_binary_property_morphological_operation(surface_topomesh, 'meristem', 0, iterations=3, method='closing')
    #topomesh_binary_property_morphological_operation(surface_topomesh, 'meristem', 0, iterations=3, method='erosion')
    


In [ ]:
layer_colormaps = dict(zip([1,2,3,4],['Blues','Reds','Greens','Greys']))

actors = []

for layer in [1,2,3,4]:
    actor = VtkActorTopomesh(surface_meshes[layer],2,property_name='meristem',property_degree=0)
    actor.update_polydata()
    
    actor.display_polydata = vtk_slice_polydata(actor.polydata,axis='x',
                                                position=(layer+1)*seg_img.extent[0]/12.,
                                                width=(layer+1)*seg_img.extent[0]/6.)
    actor.update_actor(colormap=layer_colormaps[layer],value_range=(0,1))
    actors += [actor]

    
figure_filename = dirname + '/' + filename + '/' + filename + '_layer_surfaces.png'
vtk_save_screenshot_actors(actors, figure_filename, focal_point=(0,0,-microscope_orientation), view_up=(-microscope_orientation,0,0))

vtk_display_notebook(actors)

In [ ]:
layer_height_distribution = {}
layer_height = {}
layer_height_iqr = {}
layer_distance_topomesh = {}
for layer in [2,3,4]:
    layer_topomesh = surface_meshes[layer]
    prev_layer_topomesh = surface_meshes[layer-1]
    
    layer_topomesh.update_wisp_property('eroded_meristem',0,{v:layer_topomesh.wisp_property('meristem',0)[v] for v in layer_topomesh.wisps(0)})
    #topomesh_binary_property_morphological_operation(layer_topomesh, 'eroded_meristem', 0, iterations=5, method='closing')
    topomesh_binary_property_morphological_operation(layer_topomesh, 'eroded_meristem', 0, iterations=3, method='erosion')
    layer_meristem_vertices = [v for v in layer_topomesh.wisps(0) if layer_topomesh.wisp_property('eroded_meristem',0)[v]]
    
    
    prev_layer_meristem_vertices = [v for v in prev_layer_topomesh.wisps(0) if prev_layer_topomesh.wisp_property('meristem',0)[v]]
    
    layer_points = layer_topomesh.wisp_property('barycenter',0).values(layer_meristem_vertices)
    prev_layer_points = prev_layer_topomesh.wisp_property('barycenter',0).values(prev_layer_meristem_vertices)
    
    layer_prev_layer_distances = np.linalg.norm(layer_points[:,np.newaxis] - prev_layer_points[np.newaxis],axis=2)
    
    layer_prev_layer_matching = linear_sum_assignment(layer_prev_layer_distances)
    layer_prev_layer_matching = tuple([m[layer_prev_layer_distances[layer_prev_layer_matching]<surface_matching_distance_threshold] for m in layer_prev_layer_matching])
    
    layer_distance_topomesh[layer] = edge_topomesh([[i,i+len(layer_prev_layer_matching[0])] for i in range(len(layer_prev_layer_matching[0]))],
                                                   list(layer_points[layer_prev_layer_matching[0]])+list(prev_layer_points[layer_prev_layer_matching[1]]))
    layer_distance_topomesh[layer].update_wisp_property('length',1,layer_prev_layer_distances[layer_prev_layer_matching])

    layer_height_distribution[layer-1] = layer_prev_layer_distances[layer_prev_layer_matching]
    layer_height[layer-1] = np.nanmedian(layer_prev_layer_distances[layer_prev_layer_matching])
    layer_height_iqr[layer-1] = np.nanpercentile(layer_prev_layer_distances[layer_prev_layer_matching],75) - np.nanpercentile(layer_prev_layer_distances[layer_prev_layer_matching],25)
    

In [ ]:
actors = []
for layer in [1,2,3,4]:
    actor = VtkActorTopomesh(surface_meshes[layer],2,property_name='meristem',property_degree=0)
    actor.update_polydata()
    # actor.polydata = vtk_slice_polydata(vtk_slice_polydata(actor.polydata,axis='x',position=seg_img.extent[0]/2.,width=30.),axis='y',position=seg_img.extent[1]/2.,width=30.)
    actor.display_polydata = vtk_slice_polydata(actor.polydata,axis='y',position=3*seg_img.extent[1]/5.,width=15.)
    actor.update_actor(colormap=layer_colormaps[layer],value_range=(0,1))
    actor.GetProperty().SetRepresentationToWireframe()
    actors += [actor]

    if layer>1:
        distance_actor = VtkActorTopomesh(layer_distance_topomesh[layer],1,property_name='length')
        distance_actor.line_glyph='tube'
        distance_actor.glyph_scale = 0.1
        distance_actor.update_polydata()
        # distance_actor.polydata = vtk_slice_polydata(vtk_slice_polydata(distance_actor.polydata,axis='x',position=seg_img.extent[0]/2.,width=30.),axis='y',position=seg_img.extent[1]/2.,width=30.)
        distance_actor.polydata = vtk_slice_polydata(distance_actor.polydata,axis='y',position=3.*seg_img.extent[1]/5.,width=15.)
        distance_actor.update_actor(colormap='Greys',value_range=(0,0),opacity=1)
        actors += [distance_actor]

        
figure_filename = dirname + '/' + filename + '/' + filename + '_inter_layer_distances.png'
#vtk_save_screenshot_actors([layer_actor], figure_filename, focal_point=(0,0,-microscope_orientation), view_up=(-microscope_orientation,0,0))
vtk_save_screenshot_actors(actors, figure_filename, focal_point=(0,1,0), view_up=(0,0,microscope_orientation))

vtk_display_notebook(actors)

In [ ]:
distance_data = {col:[] for col in ['layer','height']}

layer_colors = dict(zip([1,2,3,4],['b','r','g','k']))

figure = plt.figure(2)
figure.clf()
for layer in [1,2,3]:
    distance_data['height'] += [d for d in layer_height_distribution[layer]]
    distance_data['layer'] += [layer for d in layer_height_distribution[layer]]
    draw_box(figure,layer_height_distribution[layer],box_x=layer,box_width=0.33,color=layer_colors[layer],outlier_size=5)
figure.gca().set_xlim(0,4)
figure.gca().set_xticks([1,2,3])
figure.gca().set_xticklabels(['L'+str(l) for l in [1,2,3]],size=24)
figure.gca().set_ylim(0,surface_matching_distance_threshold)
figure.gca().set_ylabel("Layer height ($\mu$m)",size=24)
figure.set_size_inches(10,10)

boxplot_filename = dirname + '/' + filename + '/' + filename + '_layer_height_boxplot.png'
figure.savefig(boxplot_filename)

distance_df = pd.DataFrame(distance_data)
data_filename = dirname + '/' + filename + '/' + filename + '_layer_height_data.csv'
distance_df.to_csv(data_filename,index=False)

### Meristem cells property computation

In [ ]:
p_img.compute_image_property('volume')

l1_meristem_cells = [c for c in p_img.labels if p_img.image_property('meristem')[c] and p_img.image_property('layer')[c]==1]
l1_meristem_cell_average_volume = np.nanmean([p_img.image_property('volume')[c] for c in l1_meristem_cells])

In [ ]:
l1_labels = [c for c in p_img.labels if p_img.image_property('layer')[c] == 1]
l1_cell_points = np.array([p_img.image_property('barycenter')[c] for c in l1_labels])
surface_points = topomesh.wisp_property('barycenter',0).values(list(topomesh.wisps(0)))
l1_surface_distances = np.linalg.norm(l1_cell_points[:,np.newaxis] - surface_points[np.newaxis],axis=2)
l1_cell_surface_vertex = np.array(list(topomesh.wisps(0)))[np.argmin(l1_surface_distances,axis=1)]

for property_name in curvature_properties:
    l1_cell_curvature = dict(zip(l1_labels,topomesh.wisp_property(property_name,0).values(l1_cell_surface_vertex)))
    p_img.update_image_property(property_name,{c:l1_cell_curvature[c] if c in l1_labels else np.nan for c in p_img.labels})

In [ ]:
actors = []

cell_curvature = p_img.image_property('mean_curvature')
curvature_polydatas = image_to_vtk_cell_polydatas(seg_img, 
                                                  labels=l1_labels, 
                                                  cell_polydatas=cell_polydatas, 
                                                  cell_property=cell_curvature)
curvature_polydata = vtk_combine_polydatas([curvature_polydatas[l] for l in l1_labels])
curvature_actor = vtk_actor(curvature_polydata,colormap='RdBu_r',value_range=(-0.1,0.1))
actors += [curvature_actor]

meristem_polydata = image_to_surface_cell_scalar_property_polydata(seg_img,cell_property=cell_meristem)

contour = vtk.vtkContourFilter()
contour.SetInputData(meristem_polydata)
contour.SetValue(0,1)
contour.SetGenerateTriangles(False)
contour.Update()
meristem_contour = vtk_tube_polydata(contour.GetOutput(),radius=0.5)

meristem_actor = vtk_actor(meristem_contour,colormap=plain_colormap('red'))
actors += [meristem_actor]

figure_filename = dirname + '/' + filename + '/' + filename + '_meristem_cell_curvature.png'
vtk_save_screenshot_actors(actors, figure_filename, focal_point=(0,0,-microscope_orientation), view_up=(-microscope_orientation,0,0))
vtk_display_notebook(actors)

### Cell signal quantification

In [ ]:
if len(signal_channels) > 0:
    eroded_seg_img = labels_post_processing(seg_img, method='erosion', radius=quantification_erosion_radius, iterations=1)

    for i_s, signal_name in enumerate(signal_channels):
        signal_img = img_dict[signal_name]

        cell_total_signals = nd.sum(signal_img.get_array().astype(float),eroded_seg_img.get_array(),index=p_img.labels)
        cell_volumes = nd.sum(np.ones_like(eroded_seg_img.get_array()),eroded_seg_img.get_array(),index=p_img.labels)
        cell_signals = dict(zip(p_img.labels,cell_total_signals/cell_volumes))
        p_img.update_image_property(signal_name,cell_signals)

In [ ]:
if len(signal_channels) > 0:
    
    for i_s, signal_name in enumerate(signal_channels):
        actors = []

        signal_image_data = vtk_image_data_from_image(img_dict[signal_name].get_array(), 
                                                      voxelsize=img_dict[signal_name].voxelsize)
        signal_volume = vtk_image_volume(signal_image_data,colormap='gray')
        actors += [signal_volume]
        
        figure_filename = dirname + '/' + filename + '/' + filename + '_' + signal_name + '_signal_image.png'
        vtk_save_screenshot_actors(actors, figure_filename, focal_point=(0,0,-microscope_orientation), view_up=(-microscope_orientation,0,0))

        cell_signal = p_img.image_property(signal_name)
        signal_polydatas = image_to_vtk_cell_polydatas(seg_img, 
                                                       labels=labels, 
                                                       cell_polydatas=cell_polydatas, 
                                                       cell_property=cell_signal)

        signal_polydata = vtk_combine_polydatas([signal_polydatas[l] for l in labels])
        signal_actor = vtk_actor(signal_polydata,colormap='gray')
        actors += [signal_actor]

        meristem_actor = vtk_actor(meristem_contour,colormap=plain_colormap('red'))
        actors += [meristem_actor]

        figure_filename = dirname + '/' + filename + '/' + filename + '_meristem_cell_' + signal_name + '_signal.png'
        vtk_save_screenshot_actors(actors, figure_filename, focal_point=(0,0,-microscope_orientation), view_up=(-microscope_orientation,0,0))

        vtk_display_notebook(actors)

### Cell data export

In [ ]:
cell_df = property_image_to_dataframe(p_img)

cell_data_filename = dirname + '/' + filename + '/' + filename + '_cell_data.csv'
cell_df.to_csv(cell_data_filename,index=False)

### SAM data export

In [ ]:
meristem_data = {}
meristem_data['filename'] = [filename]
meristem_data['area'] = [meristem_area]
meristem_data['mean_curvature'] = [meristem_curvature]
meristem_data['L1_cells'] = [len(l1_meristem_cells)]
meristem_data['L1_cell_volume'] = [l1_meristem_cell_average_volume]
for layer in [1,2,3]:
    meristem_data['L'+str(layer)+'_height'] = [layer_height[layer]]
    meristem_data['L'+str(layer)+'_height_interquartile'] = [layer_height_iqr[layer]]
meristem_df = pd.DataFrame().from_dict(meristem_data)

print(meristem_df)
meristem_data_filename = dirname + '/' + filename + '/' + filename + '_meristem_data.csv'
meristem_df.to_csv(meristem_data_filename,index=False)